## Memory 

There are several use cases where it is valuable to maintain a _store_ of useful facts that can be intelligently added to the context of the agent just before a specific step. The typically use case here is a RAG pattern where a query is used to retrieve relevant information from a database that is then added to the agent's context.


AgentChat provides a {py:class}`~autogen_agentchat.memory.Memory` protocol that can be extended to provide this functionality.  The key methods are `query`, `transform`,  `add`, `clear`, and `cleanup`. 

- `query`: retrieve relevant information from the memory store 
- `transform`: mutate an agent's internal `model_context` by adding the retrieved information (used in the {py:class}`~autogen_agentchat.agents.AssistantAgent` class) 
- `add`: add new entries to the memory store
- `clear`: clear all entries from the memory store
- `cleanup`: clean up any resources used by the memory store  


## ListMemory Example

{py:class}`~autogen_agentchat.memory.ListMemory` is provided as an example implementation of the {py:class}`~autogen_agentchat.memory.Memory` protocol. It is a simple list-based memory implementation that uses text similarity matching to retrieve relevant information from the memory store. The similarity score is calculated using the `SequenceMatcher` class from the `difflib` module. The similarity score is calculated between the query text and the content text of each memory entry.   

In the following example, we will use ListMemory to similate a memory bank of user preferences and explore how it might be used in personalizing the agent's responses.

In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.memory import ListMemory, MemoryContent, MemoryMimeType
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

In [2]:
# Initialize user memory
user_memory = ListMemory()

# Add user preferences to memory
await user_memory.add(MemoryContent(content="The weather should be in metric units", mime_type=MemoryMimeType.TEXT))

await user_memory.add(MemoryContent(content="Meal recipe must be vegan", mime_type=MemoryMimeType.TEXT))


async def get_weather(city: str, units: str = "imperial") -> str:
    if units == "imperial":
        return f"The weather in {city} is 73 degrees and Sunny."
    elif units == "metric":
        return f"The weather in {city} is 23 degrees and Sunny."


assistant_agent = AssistantAgent(
    name="assistant_agent",
    model_client=OpenAIChatCompletionClient(
        model="gpt-4o-2024-08-06",
    ),
    tools=[get_weather],
    memory=[user_memory],
)

In [3]:
# Run the agent with a task.
stream = assistant_agent.run_stream(task="What is the weather in New York?")
await Console(stream)

---------- user ----------
What is the weather in New York?
---------- assistant_agent ----------
[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None, timestamp=None, source=None, score=0.463768115942029)]
---------- assistant_agent ----------
[FunctionCall(id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j', arguments='{"city":"New York","units":"metric"}', name='get_weather')]
[Prompt tokens: 128, Completion tokens: 20]
---------- assistant_agent ----------
[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j')]
---------- assistant_agent ----------
The weather in New York is 23 degrees and Sunny.
---------- Summary ----------
Number of messages: 5
Finish reason: None
Total prompt tokens: 128
Total completion tokens: 20
Duration: 0.80 seconds


TaskResult(messages=[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None, timestamp=None, source=None, score=0.463768115942029)], type='MemoryQueryEvent'), ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=128, completion_tokens=20), content=[FunctionCall(id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='assistant_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j')], type='ToolCallExecutionEvent'), ToolCallSummaryMessage(source='assistant_agent', models_usage=None, content='Th

We can inspect that the `assistant_agent` model_context is actually updated with the retrieved memory entries.  The `transform` method is used to format the retrieved memory entries into a string that can be used by the agent.  In this case, we simply concatenate the content of each memory entry into a single string.

In [4]:
await assistant_agent._model_context.get_messages()

[UserMessage(content='What is the weather in New York?', source='user', type='UserMessage'),
 SystemMessage(content='\n The following results were retrieved from memory for this task. You may choose to use them or not. :\n1. The weather should be in metric units\n', type='SystemMessage'),
 AssistantMessage(content=[FunctionCall(id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j', arguments='{"city":"New York","units":"metric"}', name='get_weather')], source='assistant_agent', type='AssistantMessage'),
 FunctionExecutionResultMessage(content=[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_OkQ4Z7u2RZLU6dA7GTAQiG9j')], type='FunctionExecutionResultMessage')]

In [5]:
result = await assistant_agent.run(task="What is the weather in New York?")
result.messages

[TextMessage(source='user', models_usage=None, content='What is the weather in New York?', type='TextMessage'),
 MemoryQueryEvent(source='assistant_agent', models_usage=None, content=[MemoryContent(content='The weather should be in metric units', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None, timestamp=None, source=None, score=0.463768115942029)], type='MemoryQueryEvent'),
 ToolCallRequestEvent(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=219, completion_tokens=20), content=[FunctionCall(id='call_YPwxZOz0bTEW15beow3zXsaI', arguments='{"city":"New York","units":"metric"}', name='get_weather')], type='ToolCallRequestEvent'),
 ToolCallExecutionEvent(source='assistant_agent', models_usage=None, content=[FunctionExecutionResult(content='The weather in New York is 23 degrees and Sunny.', call_id='call_YPwxZOz0bTEW15beow3zXsaI')], type='ToolCallExecutionEvent'),
 ToolCallSummaryMessage(source='assistant_agent', models_usage=None, content='The weather in New

We see above that the weather is returned in Centigrade as stated in the user preferences. 

Similarly, assuming we ask a separate question about generating a meal plan, the agent is able to retrieve relevant information from the memory store and provide a personalized response.

In [6]:
stream = assistant_agent.run_stream(task="Write brief meal recipe with broth")
await Console(stream)

---------- user ----------
Write brief meal recipe with broth
---------- assistant_agent ----------
[MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None, timestamp=None, source=None, score=0.5084745762711864)]
---------- assistant_agent ----------
Here's a brief vegan recipe using broth:

**Vegan Vegetable Noodle Soup**

**Ingredients:**
- 4 cups vegetable broth
- 1 cup water
- 1 cup carrots, sliced
- 1 cup celery, chopped
- 1 cup noodles (such as rice noodles or spaghetti broken into smaller pieces)
- 2 cups kale or spinach, chopped
- 2 cloves garlic, minced
- 1 tablespoon olive oil
- Salt and pepper to taste
- Lemon juice (optional)

**Instructions:**

1. **Sauté Vegetables:** In a large pot, heat olive oil over medium heat. Add minced garlic and sauté until fragrant. Add carrots and celery, and sauté for about 5 minutes, until they start to soften.

2. **Add Broth and Noodles:** Pour in the vegetable broth and water, bringi

TaskResult(messages=[TextMessage(source='user', models_usage=None, content='Write brief meal recipe with broth', type='TextMessage'), MemoryQueryEvent(source='assistant_agent', models_usage=None, content=[MemoryContent(content='Meal recipe must be vegan', mime_type=<MemoryMimeType.TEXT: 'text/plain'>, metadata=None, timestamp=None, source=None, score=0.5084745762711864)], type='MemoryQueryEvent'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=306, completion_tokens=294), content="Here's a brief vegan recipe using broth:\n\n**Vegan Vegetable Noodle Soup**\n\n**Ingredients:**\n- 4 cups vegetable broth\n- 1 cup water\n- 1 cup carrots, sliced\n- 1 cup celery, chopped\n- 1 cup noodles (such as rice noodles or spaghetti broken into smaller pieces)\n- 2 cups kale or spinach, chopped\n- 2 cloves garlic, minced\n- 1 tablespoon olive oil\n- Salt and pepper to taste\n- Lemon juice (optional)\n\n**Instructions:**\n\n1. **Sauté Vegetables:** In a large pot, heat oliv

## Custom Memory Stores (Vector DBs, etc.)

You can build on the `Memory` protocol to implement more complex memory stores. For example, you could implement a custom memory store that uses a vector database to store and retrieve information, or a memory store that uses a machine learning model to generate personalized responses based on the user's preferences etc.

Specifically, you will need to overload the  `query`, `transform`, and `add` methods to implement the desired functionality and pass the memory store to your agent.
